# A very basic oddball task demo

Hao-Ting Wang

6th June 2019

## Resources for educational perpose or general applications

 - MATLAB: [Sam's optimal MRI experiment sequence](http://www.samberens.co.uk/SamsOptseq.html)  
 - R (and general references): [Prof D Bishop's data simulation learning material](https://osf.io/skz3j/)  
 - Python: [Erik Marsja (Journal of European Psychology Students)](https://blog.efpsa.org/2016/07/12/python-programming-in-psychology-from-data-collection-to-analysis/)

## Examples with more specific purposes
Sam's projects [1](https://osf.io/rqwv5/), [2](https://osf.io/9wn3h/)  
[Hao-Ting's N-back experience sampling task](https://vcs.ynic.york.ac.uk/hw1012/nbackmindwandering)  
[Hao-Ting and Adam's go/no-go experience sampling task](https://vcs.ynic.york.ac.uk/hw1012/go_nogo_experience_sampling/)

## A very basic oddball task demo

In short, it's a go/no-go task.

Aims: 
 - understand how to use simulation for basic jittering
 - create simple simulation data
 - doing the above without actually writing an experiement (sort of)

In [1]:
import pandas as pd
import numpy as np

# set some basic variables for the task
time_length = 1.5
target = 'O'
none = 'X'
odd = 'Q'
fixation = '+'

target_chance = 0.60
none_chance = 0.20
base = 2500  # mu for fixation cross
jitter = 1000  # sigma for fixation cross

counter = 60 * time_length * 1000
n_trials = 20
t_trials = counter / n_trials  # Maximum number of trials possible

n_target = n_trials * target_chance
n_none = n_trials * none_chance
n_odd = n_trials * none_chance

In [2]:
def fixation_gitter():
    '''
    jittering fixation time
    '''
    return np.random.normal(base, jitter, 1)[0]

def exp_trials():
    '''
    Experiment trials!
    '''

    lst_trials = [target] * int(n_target) \
               + [none] * int(n_none) \
               + [odd] * int(n_odd)    
    
    np.random.shuffle(lst_trials)
    
    correct_ans = ['None' if t != target else '1' for t in lst_trials]
    
    # create output
    d = {'item': lst_trials,
         'fixation_time': [fixation_gitter() for i in range(n_trials)],
         'ans': correct_ans,
         }

    df_trials = pd.DataFrame(d)
    df_trials['stim_duration'] = t_trials - df_trials['fixation_time']

    return df_trials

In [3]:
my_trials = exp_trials()
my_trials.head()
# my_trials.to_csv('my_trials.csv')  # hint: you can save it out

,item,fixation_time,ans,stim_duration
0,X,1270.089680,None,3229.910320
1,O,4286.447259,1,213.552741
2,Q,1682.374315,None,2817.625685
3,O,1738.086275,1,2761.913725
4,X,1735.340220,None,2764.659780


Let's now image we have some fake participants.  
There are several ways to get the parameters for the simulation:
 1. Pilot data or old studies
 2. Existing pubished study (if you are using existing paradigmn)

Here today, I am just using some randome numbers.

In [4]:
mu1, sigma1 = 560, 35  # correct responses
mu2, sigma2 = 642, 54  # incorrect responses
n_sim = 20

In [5]:
def simulation(lst_trials, mu1, sigma1, mu2, sigma2):
    '''
    A hypothetical pattern of responses
    
    '''
    rt = []
    corr = []
    for item in lst_trials:
        if item=='O':
            # Let's assume they correctly response to all these trials
            fake_rt = np.random.normal(mu1, sigma1, 1)[0]
            fake_corr = 1
            
        elif item=='Q':
            # let's assume the participant press go 20% of the time
            if np.random.randint(1, 100) < 20:
                fake_rt = np.random.normal(mu2, sigma2, 1)[0]
                fake_corr = 0
            else:
                fake_rt = None
                fake_corr = 1
        else:
            # let's assume the participant press go 20% of the time
            if np.random.randint(1, 100) < 20:
                fake_rt = np.random.normal(mu2, sigma2, 1)[0]
                fake_corr = 0
            else:
                fake_rt = None
                fake_corr = 1

        rt.append(fake_rt)
        corr.append(fake_corr)
    return rt, corr

In [6]:
def exp_trials_sim(mu1, sigma1, mu2, sigma2, n_sim):
    sim_output = []
    for s in range(n_sim):
            
        lst_trials = [target] * int(n_target) \
                   + [none] * int(n_none) \
                   + [odd] * int(n_odd)
        
        np.random.shuffle(lst_trials)
        correct_ans = ['None' if t != target else '1' for t in lst_trials]
        
        # simulation of responses
        rt, corr = simulation(lst_trials, mu1, sigma1, mu2, sigma2)
            
        # create output
        d = {'item': lst_trials,
             'fixation_time': [isi() for i in range(n_trials)],
             'ans': correct_ans,
             'RT': rt,
             'correct': corr,
             }

        df= pd.DataFrame(d)
        df['stim_duration'] = t_trials - df['fixation_time']
        sim_output.append(df)   
    return sim_output

In [7]:
# Simulate
my_sim = exp_trials_sim(mu1, sigma1, mu2, sigma2, 20)

In [9]:
my_sim[2]  # Let's look at fake participant #2

,item,fixation_time,ans,RT,correct,stim_duration
0,O,3108.745160,1,579.512596,1,1391.254840
1,O,3680.681441,1,543.844119,1,819.318559
2,X,2491.132084,None,673.206628,0,2008.867916
3,X,1286.866289,None,NaN,1,3213.133711
4,Q,3369.073874,None,NaN,1,1130.926126
5,O,1673.742778,1,597.711095,1,2826.257222
6,Q,1581.445045,None,634.245139,0,2918.554955
7,O,3719.629641,1,525.122763,1,780.370359
8,O,1132.413536,1,538.069360,1,3367.586464
9,O,740.731067,1,638.249282,1,3759.268933


## Now what?

 - You can use the simulated output to do statistic tests/modeling/power analysis etc  
 - Improve your experiment design/hypothesis
 - Think about how to impliment this technique in your experiment
